In [19]:
import cv2
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
import datetime
import random
from utilities.helper_functions import helper_functions as hf
import config

In [20]:
video_path = config.remote_video_path
processed_data_path = config.successful_trial_path
reference_path = [config.marker_path + dir for dir in os.listdir(config.marker_path)]


def show_successfull_trials(video, df, offset):
     # iterate through df and for each successfull trial get the start time in seconds
    reference_image = cv2.imread(reference_path[0])
    transformation, num_match = hf.find_matching_transformation(video.read()[1], reference_path)
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new = hf.find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match or num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
            start_time_str = row['time_since_start'].split(' ')[2]
            start_time = start_time_str.split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the frames where mouse R and L break the beam
            R_time_offset = row['R']
            L_time_offset = row['L']
            
            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # display the video until the end time
            
            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                time_since_start_seconds = float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps
                time_since_start = str(datetime.timedelta(seconds=time_since_start_seconds))
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))

                cv2.putText(image, start_time_str + ' ' + time_since_start, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                # display the mice names
                cv2.putText(image, 'trial: ' + str(index), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, name1 + ' ' + name2, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, str(df['cue_times'])[:18], (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.imshow('frame', image)
                cv2.imshow('transformed', transformed)

                if cv2.waitKey(12) & 0xFF == ord('q'):
                    break

def write_successful_trials(video, df, offset, path):
    # iterate through df and for each successfull trial get the start time in seconds
    date_format = "%H:%M:%S.%f"
    reference_image = cv2.imread(reference_path[0])
    first_frame = video.read()[1]
    frame_size = first_frame.shape
    transformation, num_match, ref_path = hf.find_matching_transformation(first_frame, reference_path)
    # create video writer
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new, ref_path = hf.find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match and num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
                reference_image = cv2.imread(ref_path)
            start_time = (datetime.datetime.strptime(row['time_since_start'][7:], date_format) - datetime.datetime.strptime('00:00:00.000000', date_format)).total_seconds() - offset
            end_time = start_time + 30
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # write the video to a new file until the end time
            # check if folder exists, if not create it
            if not os.path.exists(path):
                os.makedirs(path)
            out = cv2.VideoWriter(path + '_trial_' +str(index) + '.mp4', fourcc, fps, (reference_image.shape[1], reference_image.shape[0]))
            out_full_vid = cv2.VideoWriter(path + '_trial_' +str(index) + '_full_vid.mp4', fourcc, fps, (frame_size[1], frame_size[0]))

            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                try:
                    transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))
                except:
                    print(transformation)
                    exit()
                out.write(transformed)
                out_full_vid.write(image)
            out.release()
            out_full_vid.release()

def write_all_trials(video, df, offset, path):
    # iterate through df and for each successfull trial get the start time in seconds
    date_format = "%H:%M:%S.%f"
    reference_image = cv2.imread(reference_path[0])
    first_frame = video.read()[1]
    frame_size = first_frame.shape
    transformation, num_match, ref_path = hf.find_matching_transformation(first_frame, reference_path)
    # create video writer
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for index, row in df.iterrows():
        transformation_new, num_match_new, ref_path = hf.find_matching_transformation(video.read()[1], reference_path)
        if num_match_new > num_match and num_match_new > 30:
            transformation = transformation_new
            num_match = num_match_new
            reference_image = cv2.imread(ref_path)
        start_time = (datetime.datetime.strptime(row['time_since_start'][7:], date_format) - datetime.datetime.strptime('00:00:00.000000', date_format)).total_seconds() + offset
        end_time = start_time + 30
        # set the video to the start time
        fps = video.get(cv2.CAP_PROP_FPS)
        current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
        time_offset = start_time - current_time
        for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
            video.read()

        # find the mice names
        name1 = row['names'][:2]
        name2 = row['names'][2:]
        # write the video to a new file until the end time
        # check if folder exists, if not create it
        if not os.path.exists(path):
            os.makedirs(path)
        out = cv2.VideoWriter(path + '_trial_' +str(index) + '.mp4', fourcc, fps, (reference_image.shape[1], reference_image.shape[0]))
        out_full_vid = cv2.VideoWriter(path + '_trial_' +str(index) + '_full_vid.mp4', fourcc, fps, (frame_size[1], frame_size[0]))

        while True:
            success, image = video.read()
            if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                print(success)
                break
            try:
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))
            except:
                print(transformation)
                exit()
            out.write(transformed)
            out_full_vid.write(image)
        out.release()
        out_full_vid.release()



In [21]:
# get list of video names from the folder of processed CSVs
csv_times = [ hf.process_time_string(name.split('_')[0]) for name in os.listdir(processed_data_path)]
csv_paths = [name for name in os.listdir(processed_data_path)]
# iterate through each csv and find the closest video name
video_times = [hf.process_time_string(name.split('_')[0]) for name in os.listdir(video_path)]
video_paths = [name for name in os.listdir(video_path)]
csv_vid_match = dict()
for csv_time in csv_times:
    closest_time = video_times[0]
    closest_path = video_paths[0]
    for i, video_time in enumerate(video_times):
        if abs((csv_time - video_time).total_seconds()) <  abs((csv_time - closest_time).total_seconds()):
            closest_time = video_time
            closest_path = video_paths[i]
    total_delta = (csv_time - closest_time).total_seconds()
    if total_delta < 10:
        csv_vid_match[csv_paths[csv_times.index(csv_time)]] = {
            'video_name': closest_path,
            'delta_seconds': total_delta
        }


significant_trials = []
for csv_name in csv_vid_match.keys():
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) >= 10:
        significant_trials.append(csv_name)
significant_trials.sort()
significant_trials.reverse()
# randoize order
random.shuffle(significant_trials)


    

In [22]:
# iterate throught each video and load the corresponding video and csv and then write the video with the successful trials
for csv_name in significant_trials:
    # check if folder for trial exists, if so skip
    if os.path.exists(config.video_output_path + csv_name.split('_')[0] + '/') or os.path.exists(config.video_output_path[:-1] + '/' + csv_name.split('_')[0] + '/'):
        continue
    video = cv2.VideoCapture(video_path + csv_vid_match[csv_name]['video_name'])
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) < 10:
        continue
    write_successful_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], config.video_output_path + csv_name.split('_')[0] + '/')
    print(csv_name)
    write_all_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], config.video_output_path[:-1] + '_all/' + csv_name.split('_')[0] + '/')


20230822152926_log.csv
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tr

In [23]:
# for csv_name in significant_trials:
#     if csv_name[5] != '8':
#         continue
#     video = cv2.VideoCapture(video_path + csv_vid_match[csv_name]['video_name'])
#     df = pd.read_csv(processed_data_path + '/' + csv_name)
#     #write_successful_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], config.video_output_path + csv_name.split('_')[0] + '/')
#     print(csv_name)
#     write_all_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], config.video_output_path[:-1] + '_all/' + csv_name.split('_')[0] + '/')

In [24]:
significant_trials

['20230721145114_log.csv',
 '20230822152926_log.csv',
 '20230817105415_log.csv',
 '20230810161151_log.csv',
 '20230721122412_log.csv',
 '20230714161545_log.csv',
 '20230721161826_log.csv',
 '20230830140911_log.csv',
 '20230812170534_log.csv',
 '20230814102821_log.csv',
 '20230720104701_log.csv',
 '20230720141801_log.csv',
 '20230831104835_log.csv',
 '20230815163010_log.csv',
 '20230718151456_log.csv',
 '20230712153221_log.csv',
 '20230818121524_log.csv',
 '20230823103645_log.csv',
 '20230713094734_log.csv',
 '20230824111456_log.csv',
 '20230901101428_log.csv',
 '20230718144045_log.csv',
 '20230829113352_log.csv',
 '20230712104541_log.csv',
 '20230816103729_log.csv',
 '20230718154037_log.csv',
 '20230719100754_log.csv',
 '20230810171031_log.csv',
 '20230712151237_log.csv',
 '20230814161151_log.csv',
 '20230712161638_log.csv',
 '20230812150714_log.csv',
 '20230714140922_log.csv',
 '20230718161229_log.csv',
 '20230814101104_log.csv',
 '20230714114827_log.csv',
 '20230721100255_log.csv',
 

In [25]:
config.video_output_path[:-1] + '_all/'

'/mnt/teams/Tsuchitori/social_interaction_trials_all/'

In [26]:
for index, row in pd.read_csv(processed_data_path +  '20230824111456_log.csv').iterrows():
    date_format = "%H:%M:%S.%f"
    time = datetime.datetime.strptime(row['time_since_start'][7:], date_format) - datetime.datetime.strptime('00:00:00.000000', date_format)
    print(time.total_seconds())

0.000718
76.00088
153.000673
229.000235
305.00041
381.000591
427.000797
474.000313
550.000197
626.000364
702.000528
778.000711
854.000899
916.000122
962.000383
1038.000584
1114.000757
1190.000941
1267.000131
1343.000308
1419.000487
1465.000747
1541.000921
1618.000793
1664.000267
1740.000425
1816.000612
1877.000835
1954.000212
2030.000176
2091.000395
2167.000591
2243.000759
2289.000959
2366.000151
2442.000338
2518.000519
2579.000741
2655.00092
2732.000895
2808.00026
2884.000436
2960.000601
3036.000772
3112.000949
3189.000131
3265.000313
3341.000492
3417.000677
3493.000846
3570.000335
3646.000182
3692.000381
3768.000561
3844.000747
3905.000963
3967.000184
4013.000386
4089.000564
4165.000736
4241.000916
4318.000112
4394.000298
4470.000497
4531.000742
4592.000986
4639.000251
4715.000449
4776.000674
4852.000846
4929.000388
5005.000227
5081.000415
5157.000602
5203.000813
5279.000998
5356.000166
5432.000337
5508.000535
5584.000736
5660.000917
5737.000106
5813.000299
5889.000496
5965.000687
60